In [1]:
import pandas as pd

In [2]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bi

In [7]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120493 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [3]:
scores = pd.read_csv('all_scores.csv')

In [22]:
scores

,client_id,product,channel,score,optimal_decision
0,0,credit card,sms,0.001995,0.0
1,0,credit,call,0.921569,1.0
2,0,credit,sms,0.010591,0.0
3,0,credit card,call,0.017675,0.0
4,1,credit card,call,0.017675,0.0
...,...,...,...,...,...
79995,19998,credit card,sms,0.001995,0.0
79996,19999,credit,sms,0.102331,1.0
79997,19999,credit card,sms,0.008074,0.0
79998,19999,credit,call,0.009653,0.0


In [4]:
CHANNELS_LIMITS = {
    'call': 4000,
    'sms': 7000
}

In [18]:
import pyomo.environ as pyo

def optimize(frame, channel_limits):
    df = frame.copy()

    model = pyo.ConcreteModel()

    model.x = pyo.Var(range(df.shape[0]), within=pyo.Binary)
    df['x'] = [model.x[i] for i in range(df.shape[0])]

    model.objective = pyo.Objective(expr=sum(df.loc[i, 'score'] * df.loc[i, 'x'] for i in range(df.shape[0])), sense=pyo.maximize)

    def channel_constraint_rule(model, channel):
        return sum(df.loc[i, 'x'] for i in range(df.shape[0]) if df.loc[i, 'channel'] == channel) <= channel_limits[channel]
    model.channel_constraints = pyo.ConstraintList()
    for channel in channel_limits:
        model.channel_constraints.add(channel_constraint_rule(model, channel))

    def client_constraint_rule(model, client_id):
        group = df[df['client_id'] == client_id]
        return sum(model.x[i] for i in range(group.shape[0])) <= 1
    model.client_constraints = pyo.ConstraintList()
    for client_id in df['client_id'].unique():
        model.client_constraints.add(client_constraint_rule(model, client_id))

    solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
    results = solver.solve(model, tee=True)

    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        optimal_proposals = [pyo.value(model.x[i]) for i in range(df.shape[0])]
        return optimal_proposals
    else:
        print("No feasible solution found.")
    print(results)
    # model.pprint()
    # model.display()

    del df

In [19]:
optimal_decisions = optimize(frame=scores, channel_limits=CHANNELS_LIMITS)

Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpfaeb97nv.pyomo.lp -stat=1 -solve -solu /tmp/tmpfaeb97nv.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 3 (-19999) rows, 79938 (-62) columns and 79942 (-80058) elements
Statistics for presolved model
Original problem has 80000 integers (80000 of which binary)
Presolved problem has 79938 integers (79938 of which binary)
==== 0 zero objective 20081 different
==== absolute objective values 20081 different
==== for integers 0 zero objective 20081 different
==== for integers absolute objective values 20081 different
===== end objective counts


Problem has 3 rows, 79938 columns (79938 with objective) and 79942 elements
There are 79934 singletons with objective 
Column brea

In [21]:
scores['optimal_decision'] = optimal_decisions

#распределение продуктов в каналах
scores[scores['optimal_decision']==1].groupby(['channel', 'product']).\
                                    agg({'client_id': 'count'}).\
                                    rename(columns={'client_id': 'client_cnt'})

client_cnt
channel product                
call    credit             1613
        credit card        2387
sms     credit             2996
        credit card        4004

**Для сравнения результаты решения с MIP**

    call	credit	     1724

            credit card	2276

    sms	  credit	    2807

            credit card	4193